In [1]:
# import packages
import gc
import glob
import cmcrameri.cm as ccm
import matplotlib as mpl
from matplotlib import colors
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from params import savepath, homedir, allnames, datadir

plt.style.use('science.mplstyle')

states = gpd.read_file('../data/ne_110m_admin_1_states_provinces.shp')

In [2]:
domains = ['d01', 'd02', 'd03', 'd04']
coordsdict = {}
for domain in domains:
    coordsdict[domain] = xr.open_dataset(f'/glade/campaign/uwyo/wyom0112/postprocess/meta/meta_new/wrfinput_{domain}')

In [3]:
params = {'vars': ['prec','t2','snow'],
          'states' : ['CA', 'WY'],
          'res': ['45','9','3']}
res_domain = {'3': ['d03','d04'],
               '9' :['d02'],
               '45':['d01']}
domain_state = {'d01':['CA','WY'],
                'd02':['CA','WY'],
                'd03':['CA'],
                'd04':['WY'],}
boundaries = {'d01':{'CA': [43,60,35,68],
                     'WY': [68,81,43,63]},
              'd02':{'CA': [44,125,87,251],
                     'WY': [165,230,130,227]},
              'd03':{'CA': [0,-1,0,-1]},
              'd04':{'WY': [0,-1,0,-1]}}
state_res_domain = {'CA': {'45':'d01',
                            '9': 'd02',
                            '3':'d03'},
                     'WY': {'45':'d01',
                            '9': 'd02',
                            '3':'d04'},}

gcm = 'ec-earth3-veg'
bc = 'BC'

In [4]:
%%time

## make the dictionary: 
alldata = {}
for var in params['vars']: 
    alldata[var] = {}
    for state in params['states']:
        alldata[var][state] = {}
        for res in params['res']:
            alldata[var][state][res] = {}

## fill raw data
for var in params['vars']: 
    for state in params['states']:
        for res in params['res']:
            domain = state_res_domain[state][res]
            alldata[var][state][res]['raw'] = xr.open_dataset(f'/glade/campaign/uwyo/wyom0112/berkeley/{var}_{gcm}_{domain}_{bc}.nc')
            if var == 'snow':
                alldata[var][state][res]['annual_max'] = alldata[var][state][res]['raw'].resample(time = '1AS').max()
            elif var == 't2':
                tmp = alldata[var][state][res]['raw'] - 273 ##  .resample(time = '1AS')
                mask = tmp > 0
                alldata[var][state][res]['annual_max'] = tmp.where(mask).resample(time = '1AS').sum()
            else: 
                alldata[var][state][res]['annual_max'] = alldata[var][state][res]['raw'].resample(time = '1AS').sum()
                

            left, right, bottom, top = boundaries[domain][state]
            alldata[var][state][res]['annual_max_state'] =  alldata[var][state][res]['annual_max'].sel(lat2d = slice(bottom, top),
                                                                   lon2d = slice(left, right))

CPU times: user 2min 45s, sys: 11min 7s, total: 13min 52s
Wall time: 41min 14s


In [5]:
alldata_to_save = {}
for var in params['vars']: 
    alldata_to_save[var] = {}
    for state in params['states']:
        alldata_to_save[var][state] = {}
        for res in params['res']:
            alldata_to_save['annual_max_clipped'] = alldata[var][state][res]['annual_max_state']

In [6]:
np.save('/glade/campaign/uwyo/wyom0112/berkeley/annual_max_clipped.npy',alldata_to_save)

In [7]:
%%time
## partitioning:
partition = {}
for state in params['states']:
    partition[state] = {}
    for res in params['res']:
        domain = state_res_domain[state][res]
        temp = alldata['t2'][state][res]['raw'] 
        prec = alldata['prec'][state][res]['raw']
        mask = temp['t2'] > 273.15
        rainfall = prec.where(mask)
        snowfall = prec.where(~mask)
        partition[state][res] = {}
        partition[state][res]['snowfall'] = snowfall
        partition[state][res]['rainfall'] = rainfall
        

CPU times: user 1min 4s, sys: 8min 11s, total: 9min 15s
Wall time: 46min 3s


In [8]:
partition_annual = {}
for state in params['states']:
    partition_annual[state] = {}
    for res in params['res']:
        partition_annual[state][res] = {}
        partition_annual[state][res]['snowfall'] = partition[state][res]['snowfall'].resample(time = '1AS').sum()
        partition_annual[state][res]['rainfall'] = partition[state][res]['rainfall'].resample(time = '1AS').sum()

In [9]:
np.save('/glade/campaign/uwyo/wyom0112/berkeley/partition_annual.npy',partition_annual)

In [10]:
coords_clipped = {}
for state in params['states']:
    coords_clipped[state] = {}
    for res in params['res']:
        domain = state_res_domain[state][res]
        left, right, bottom, top = boundaries[domain][state]
        basecoords = coordsdict[state_res_domain[state][res]]
        coords_clipped[state][res] =  basecoords.sel(south_north = slice(bottom, top),
                                                               west_east = slice(left, right))

In [13]:
np.save('/glade/campaign/uwyo/wyom0112/berkeley/coords_clipped.npy',coords_clipped)

In [11]:
snowmasks = {}
for state in params['states']:
    snowmasks[state] = {}
    for res in params['res']:
        snowdata = alldata['snow'][state][res]['annual_max_state']['snow']
        mask = np.nansum(snowdata[0:20], axis = 0) > 400
        snowmasks[state][res] = mask

In [12]:
np.save('/glade/campaign/uwyo/wyom0112/berkeley/snowmasks.npy',snowmasks)